In [1]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from dwave.plugins.qiskit import DWaveMinimumEigensolver
import numpy as np

# Convert integer linear program from qiskit and solve with default Solver Advantage_system1.1
Problem ID : c257837d-8fbb-4694-836a-3c30b6f2e0e1

In [2]:
A = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,5,2,3,1,1],
              [0,0,1,2,2,0]])
m, n = A.shape
c =  np.array([-1,2,-2,2,1,1])
b = np.array([6,8,5,2])

solution = np.array([2,0,0,0,1,2])

print(all(A.dot(solution) == b))

True


# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in N^n$ with:

$\min c^T x $ with constraint $Ax=b$ 

In [3]:
model = QuadraticProgram("Integer Test")
lowerbound = 0
upperbound = 3 # for values greater than 7 we probably can't find a solution

# add variables
x = [model.integer_var(lowerbound = lowerbound, upperbound = upperbound, name = 'x'+str(j)) for j in range(n)]

# add linear constraints
for i in range(m):
    model.linear_constraint(linear=A[i], sense='==', rhs=b[i], name='A'+str(i)+'eq'+'b'+str(i))
    
# add linear objective function

model.minimize(linear = c)

In [4]:
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Integer Test

Minimize
 obj: - x0 + 2 x1 - 2 x2 + 2 x3 + x4 + x5
Subject To
 A0eqb0: x1 + 2 x2 + 3 x3 + 4 x4 + x5 = 6
 A1eqb1: 2 x0 + x1 + 4 x2 + x3 + 2 x5 = 8
 A2eqb2: x0 + 5 x1 + 2 x2 + 3 x3 + x4 + x5 = 5
 A3eqb3: x2 + 2 x3 + 2 x4 = 2

Bounds
       x0 <= 3
       x1 <= 3
       x2 <= 3
       x3 <= 3
       x4 <= 3
       x5 <= 3

Generals
 x0 x1 x2 x3 x4 x5
End



In [5]:
# Use D-Wave QPU as a minimum eigen solver
# See https://github.com/dwavesystems/dwave-qiskit-plugin

dwave_solver = DWaveMinimumEigensolver()
optimizer = MinimumEigenOptimizer(dwave_solver)
result = optimizer.solve(model) 

In [6]:
print(result)

optimal function value: 1.0
optimal value: [2. 0. 0. 0. 1. 2.]
status: SUCCESS


In [7]:
result.min_eigen_solver_result.sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,chain_break_fraction,energy,num_occurrences
0,0,1,0,0,0,0,0,0,1,0,0,1,0.0,-14941.5,1
1,0,1,0,0,1,0,0,0,1,0,0,0,0.0,-14917.5,2
2,0,0,0,0,1,0,1,0,0,0,1,0,0.0,-14857.5,3
3,0,0,0,0,0,1,0,0,0,0,0,0,0.0,-14806.5,1
4,0,0,0,0,0,1,0,0,0,0,1,0,0.0,-14805.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0,1,0,0,1,0,0,0,0,0,1,1,0.0,-13767.5,1
62,0,0,0,0,0,0,0,0,1,0,1,0,0.0,-13652.5,1
63,0,0,1,0,0,0,0,0,0,0,1,0,0.0,-13651.5,1
64,0,1,1,0,0,0,0,0,1,0,1,1,0.0,-13566.5,1
